In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure the plotting routines
import pandas as pd

# Import your CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Jael2024"

# Connect to the database via CRUD Module
db = AnimalShelter(username, password, "nv-desktop-services.apporto.com", 31702, "AAC", "animals")

# Query all documents from MongoDB (initial unfiltered query)
df = pd.DataFrame.from_records(db.read({}))

# MongoDB returns the '_id' column, which has an ObjectID that can cause issues with display
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load the Grazioso Salvare logo from the local directory and encode it
image_filename = 'GraziosoSalvareLogo.png'  # Local path for the image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Create app layout
app.layout = html.Div([
    # Grazioso Salvare logo with a URL anchor tag to the homepage
    html.A(href='https://www.snhu.edu', children=[
        html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', height='100px'))
    ]),
    html.Center(html.H1('CS-340 Dashboard - Jael Andre')),  # Unique identifier with my name
    html.Hr(),
    
    # Interactive filter options (radio buttons)
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water_rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster_rescue'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',  # Default value
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    
    # Data table to display filtered data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    
    # Div for graphs and map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),  # Placeholder for pie chart (or empty pie chart for reset)
        html.Div(id='map-id', className='col s12 m6'),    # Placeholder for map
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update the data table based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Default query (all data)
    query = {}
    
    # Define filters for each rescue type
    if filter_type == 'water_rescue':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif filter_type == 'mountain_rescue':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif filter_type == 'disaster_rescue':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
    elif filter_type == 'reset':
        query = {}  # Reset to all data

    # Query the database using the filter
    data = pd.DataFrame.from_records(db.read(query))
    data.drop(columns=['_id'], inplace=True)  # Remove '_id' column
    return data.to_dict('records')

# Display pie chart for all filters; show an empty pie chart for 'Reset'
@app.callback(
    Output('graph-id', 'children'),
    [Input('filter-type', 'value'), Input('datatable-id', 'derived_virtual_data')]
)
def update_graph(filter_type, viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    if filter_type == 'reset':
        # Return an empty pie chart when "Reset" is selected
        fig = px.pie(values=[], names=[], title="No Data Available")
    else:
        # Create a pie chart for filtered data
        if not dff.empty:
            fig = px.pie(dff, names='breed', title='Breed Distribution (Filtered)')
        else:
            fig = px.pie(values=[], names=[], title="No Data Available")

    return dcc.Graph(figure=fig)

# Update the geolocation chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0 or index is None:
        # Return a default map if no data is available
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    
    # Extract lat/long if available
    lat = dff.iloc[row].get('location_lat', 30.75)
    lon = dff.iloc[row].get('location_long', -97.48)
    breed = dff.iloc[row].get('breed', 'Unknown')
    name = dff.iloc[row].get('name', 'Unknown')

    # Return the map with a marker for the selected row's location
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]

# Run app and display result inline in the notebook
app.run_server(debug=True)


Dash app running on http://127.0.0.1:16293/
